In [1]:
import pandas as pd
import numpy as np

from pandas_profiling import ProfileReport
import plotly.express as px
import re

from itertools import groupby
from datetime import datetime
import math

#import re
#from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer

#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize




import seaborn as sns

import zipfile

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 50

In [2]:
zf = zipfile.ZipFile('../data/raw/Somerset Sold Homes .csv.zip') 
df = pd.read_csv(zf.open('Somerset Sold Homes .csv'))
df.head(2)

,Address,City,State,Zip,property-href,Sale Price,Redfin Estimate,Beds,Baths,Size,Listing Agent,Listing Broker,Buyers Agent,Buyers Broker,Fact 1,Fact 2,Fact 3,Fact 4,Fact 5,Fact 6,Description,Built,Pricing Insights,Details,Amenities,Comments,Price History,Public Facts,Full Details,School,Walk Score,Transit Score,Bike Score,Similar Homes 1,Similar Home 2,Similar Home 3,Similar Home 4,Similar Home 5,Similar Home 6,Similar Home 7,Similar Home 8,Similar Home 9,Recently Sold 1,Recently Sold 2,Recently Sold 3,Price Insight 1,Price Insight 2,Price/SF,Price Insight 4,Home Status,Home Fact 2,Home Fact 3,Home Fact 4,Home Fact 5,Home Fact 6,Home Fact 7
0,170 JEFFERSON Ave,North Plainfield,NJ,07060,https://www.redfin.com/NJ/North-Plainfield/172...,"$140,000","$304,207",NaN,NaN,"2,328",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1924,"Redfin Estimate$304,207Price/Sq.Ft.$131","StatusSoldYear Built1924Lot Size6,625 Sq. Ft.",NaN,NaN,"TodayDec 11, 2019DateSold (Public Records)Publ...","Beds—Baths—Sq. Ft.2,328Stories2Lot Size6,625 S...",NaN,This home is within the North Plainfield Boro...,59 / 100,33 / 100,44 / 100,"Get $4,563 More Selling Your Home with a Redfi...","$319,900—Beds—Baths—Sq. Ft.241 Orange St, Dune...","$392,0003 Beds3.5 Baths—Sq. Ft.15 WEBER Ave, H...","NEW 10 HRS AGO$350,0005 Beds2 Baths1,543 Sq. F...","$435,000—Beds—Baths—Sq. Ft.417 Chestnut St, Mi...","$339,900—Beds—Baths1,946 Sq. Ft.205 N 3rd Ave,...","$299,0005 Beds2 Baths—Sq. Ft.137 Grove St, Nor...","$365,0006 Beds2 Baths1,690 Sq. Ft.210 FILAK Av...","$599,0004 Beds3.5 Baths—Sq. Ft.8 ROUND TOP Rd,...","SOLD SEP 28, 2020$570,000 Last Sold Price —Bed...","SOLD MAY 1, 2020$455,000 Last Sold Price —Beds...","SOLD JUL 16, 2020$110,000—Beds—Baths2,754 Sq. ...","$304,207",NaN,$131,NaN,Sold,"Lot Size6,625 Sq. Ft.",NaN,NaN,Year Built1924,NaN,NaN
1,144 Countryside Dr Unit 4F,Basking Ridge,NJ,07920,https://www.redfin.com/NJ/Basking-Ridge/144-Co...,"$200,000","$205,617",NaN,NaN,892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1983,NaN,NaN,Taxes / AssessmentsAssessor InformationAssessm...,NaN,"TodayFeb 21, 2020DateSold (Public Records)Publ...",Beds—Baths—Sq. Ft.892Stories—Lot Size—StyleCon...,Taxes / AssessmentsAssessor InformationAssessm...,NaN,16 / 100,NaN,22 / 100,"$375,0003 Beds1 Bath—Sq. Ft.160 MOUNT AIRY Rd,...","$375,0003 Beds1 Bath—Sq. Ft.160 MOUNT AIRY Rd,...","$679,0005 Beds3 Baths—Sq. Ft.9 BLAZURE Dr, Ber...","$509,0003 Beds2 Baths1,568 Sq. Ft.134 W OAK St...","$629,0004 Beds2.5 Baths—Sq. Ft.22 MAPLE VILLAG...","$279,9002 Beds2 Baths—Sq. Ft.232 English Pl, B...","$450,0003 Beds1.5 Baths—Sq. Ft.77 MINE MOUNT R...","$469,9003 Beds2 Baths1,759 Sq. Ft.11 Chilton S...","$739,9005 Beds2.5 Baths—Sq. Ft.58 MT AIRY Rd, ...","SOLD AUG 31, 2020$525,000 Last Sold Price 3 Be...","SOLD JUN 8, 2020$334,000 Last Sold Price 4 Bed...","SOLD SEP 4, 2020$440,000 Last Sold Price 3 Bed...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
## Read the web scrapped file
##df=pd.read_csv('../data/raw/Somerset Sold Homes .csv.zip', compression='zip', header=0)
##df.head(2)

In [4]:
## Remove rows with unknown address
df = df[-(df.Address.isin(['Unknown Address']))]
df.shape

(4441, 56)

#### Dropping rows/columns and resetting the index

In [5]:
## List all columns and how many null values they have
##df.apply(lambda x: np.sum(x.isnull()))
drp_thrshld = len(df)*.7
drp_thrshld

3108.7

In [6]:
x = df.apply(lambda x: np.sum(x.isnull()))
df.drop(x[x > drp_thrshld].index, axis=1, inplace=True)

df = df[-df.City.isnull()]
df = df[-df.Address.isnull()]
df = df[-df.Zip.isnull()]

df.shape
##df.apply(lambda x: np.sum(x.isnull()))

(4152, 44)

In [7]:
## Identify if the listed property is a condo and remove them
is_condo = ['Y' if ('Condo' in str(x)) else 'N' for x in df['Public Facts']]
df['is_condo'] = is_condo
df = df[df.is_condo == 'N']
df.shape

(3472, 45)

In [8]:
len(np.unique(df['property-href']))

3472

In [9]:
df.drop_duplicates(subset = ['property-href'], inplace=True)
df.shape

(3472, 45)

In [10]:
df.reset_index(inplace=True, drop=True)

In [11]:
##df.apply(lambda x: np.sum(x.isnull()))

### General Search

In [12]:
#x = df.apply(lambda col: np.sum(['Upgrade' in str(x) for x in col]))
##x = df.apply(lambda col: np.sum(['Condo' in str(x) for x in col]))
#x[x>0]

In [13]:
##[x.split('Upgrade')[1][0:100] for x in df.Amenities if 'Upgrade' in str(x)]

### The following section scans through the "Amenities" columns and looks for information on the property. This is a process of generating addtional features from unstructured data

In [14]:
f_wood = []
f_tile = []
f_carpet = []
f_laminate = []
floor_upg_sts = []

stl_shr = []

#grmt_kit = []
#kit_ilnd = []
#w_d_bar = []
kitchen_upg_sts = []

house_int_upg_sts = []
house_ext_upg_sts = []

#rcs_ltng = [] 
#sky_lt = []
#crn_mldg = []
#fireplace = []

sprnklr_s = []
fencing = []
twn_house = []
deck_patio = []

bsmt = []
stories = []

##Basement: Finished, Full, Recreation Room
### of Stories: 1
### Living Sq. Ft:

s_cols = [x for x in df.columns if 'Amenities' in x ]
print(s_cols)

for cols in s_cols:
    print(cols)
    
    df[cols].fillna('MI', inplace = True)
    
    for i, itm in enumerate(df[cols]):
        
        #grmt_kit.append('Y') if ('Gourmet' in itm) else grmt_kit.append('N')
        #kit_ilnd.append('Y') if ('Island' in itm) else kit_ilnd.append('N')
        #w_d_bar.append('Y') if ('Wet/Dry Bar' in itm) else w_d_bar.append('N')
        #rcs_ltng.append('Y') if ('Recessed Lighting' in itm) else rcs_ltng.append('N')
        #sky_lt.append('Y') if ('Skylight' in itm) else sky_lt.append('N')
        #crn_mldg.append('Y') if ('Skylight' in itm) else crn_mldg.append('N')
        
        #Bathroom Upgrade
        stl_shr.append('Y') if ('Stall Shower' in itm) else stl_shr.append('N')
        
        ## Internal Upgrade
        if (('Recessed Lighting' in itm) | ('Skylight' in itm) | ('Skylight' in itm)):
            house_int_upg_sts.append('Y')
        else:
            house_int_upg_sts.append('N')
        
        ## External Upgrade
        if (('Deck' in itm) | ('Patio' in itm) | ('Sprinkler' in itm) | ('Fencing' in itm)):
            house_ext_upg_sts.append('Y')
        else:
            house_ext_upg_sts.append('N')
         
        #Kitchen Upgrade
        if (('Countertop' in itm) | ('Gourmet' in itm) | ('Island' in itm) | ('Wet/Dry Bar' in itm) |
            ('Granite/Corian' in x) | ('Upgraded' in itm.split('Countertop')[0][-15:])):
            kitchen_upg_sts.append('Y')
        else:
            kitchen_upg_sts.append('N')
        
        #Floor Upgrade
        if ('Flooring: ' in itm):
            x = itm.split('Flooring: ')[1].split('Interior Features')[0][0:50]
            f_wood.append('Y') if 'Wood' in x else f_wood.append('N') 
            f_tile.append('Y') if ('Marble' in x or 'Tile' in x) else f_tile.append('N')
            f_carpet.append('Y') if ('Carpet' in x) else f_carpet.append('N') 
            f_laminate.append('Y') if ('Laminate' in x or 'Vinyl' in x) else f_laminate.append('N')
        else:
            f_wood.append('UNK')
            f_tile.append('UNK')
            f_carpet.append('UNK')
            f_laminate.append('UNK')
            
        floor_upg_val = ((f_wood[i] == 'Y')*2 + (f_tile[i] == 'Y')*2 + 
                          (f_carpet[i] == 'Y')*1 + (f_laminate[i] == 'Y')*1)
        
        floor_upg_sts.append(['Y' if floor_upg_val >=3 else 'N'])
        
        ##Basement Upgrade
        if ('Basement Information' in itm):
            x = itm.split('Basement Information')[1]
            if ('Finished' in x):
                bsmt.append('FN')
            else:
                bsmt.append('UF')
        else:
            bsmt.append('NB')
        
        #Other Information
        twn_house.append('Y') if ('Townhouse' in itm) else twn_house.append('N')
       
        if ('# of Stories: ' in itm):
            x = itm.split('# of Stories: ')
            stories.append(re.sub('[^0-9.]', '', x[1][0:4]))
        else:
            stories.append('UNK')
        

['Amenities']
Amenities


In [15]:
##[x for x in df['Full Details'] if 'wood' in str(x)][0:10]

#### Zip Code Cleanup and Split

In [16]:
## Zip Code Cleanup

def z_splitter(x, ix):
    
    if x is None:
        return("MI")
    
    x = str(x)
        
    if ix == 1: 
        x = x.split("-")[0]
        if len(x) == 5:
            return(x)
        else:
            return('0' + x)
    else:
        if len(x.split("-")) > 1:
            return(x.split("-")[1])
        else:
            return("MI")

Zip_5 = df.Zip.apply(lambda x: z_splitter(x, 1))
Zip_E = df.Zip.apply(lambda x: z_splitter(x, 2))
                     

In [17]:
np.sum([x=='MI' for x in Zip_5])

0

In [18]:
## General Search
#x = df.apply(lambda col: np.sum(['Granite' in str(x) for x in col]))
#x[x>0]

#### Extracting MLS

In [19]:
##Finding all columns containing MLS# and extract MLS# from them

x = df.apply(lambda col: np.sum(['MLS #' in str(x) for x in col]))

mrg = df[x[x>0].index].apply(lambda x: ''.join(str(x)), axis=1)

MLS = [re.findall("MLS #[0-9]+", str(x), re.IGNORECASE)[0] 
     if re.findall("MLS #[0-9]+", str(x), re.IGNORECASE) else 'MI'
     for x in mrg]

MLS[0:4]


['MI', 'MLS #3', 'MI', 'MI']

In [20]:
print(np.sum([x=='MI' for x in MLS]), len(np.unique(MLS)))

2618 186


#### Cleaning up the Bathrooms

In [21]:
##del baths
baths = []
df.Baths.fillna('MI', inplace=True)
df['Public Facts'].fillna('MI', inplace=True)

for i, itm in enumerate(df.Baths):
    
    if itm != 'MI':
        baths.append(itm)
    else:
        ##print(i)
        if df['Public Facts'][i] != 'MI' and len(df.loc[i, 'Public Facts'].split('Baths')) > 1:

            if df.loc[i, 'Public Facts'].split('Baths')[1].split('Sq. Ft.')[0] != "—":
                ##print(df.loc[i, 'Public Facts'].split('Baths')[1].split('Sq. Ft.')[0] )
                baths.append(df.loc[i, 'Public Facts'].split('Baths')[1].split('Sq. Ft.')[0])
            else:
                baths.append('MI')
        else:
            baths.append('MI')
        
print(np.sum([x=='MI' for x in baths]), len(np.unique(baths)))

1575 14


#### Getting the Bedrooms

In [22]:
beds = []
df.Beds.fillna('MI', inplace=True)
df['Public Facts'].fillna('MI', inplace=True)

for i, itm in enumerate(df.Beds):
    
    if itm != 'MI':
        beds.append(itm)
    else:
        if df['Public Facts'][i] != 'MI':

            if df.loc[i, 'Public Facts'].split('Beds')[1].split('Baths')[0] != "—":
                ##print(df.loc[i, 'Public Facts'].split('Baths')[1].split('Sq. Ft.')[0] )
                beds.append(df.loc[i, 'Public Facts'].split('Beds')[1].split('Baths')[0])
            else:
                beds.append('MI')
        else:
            beds.append('MI')
        
print(np.sum([x=='MI' for x in beds]), len(np.unique(beds)))

1593 10


#### Finding the age of the house based on the year it was built

In [23]:
## Compute the age of the current year
this_year = datetime.today().year

age = [re.findall("[0-9]{4}", str(x), re.IGNORECASE)[0] 
     if re.findall("[0-9]{4}", str(x), re.IGNORECASE) else ''
     for x in df.Built]

age = [this_year - int(x) if x != '' else 100 for x in age]

print(np.sum([x=='MI' for x in age]), len(np.unique(age)))

0 147


In [24]:
##x = df.apply(lambda col: np.sum(['Lot Size' in str(x) for x in col]))
##x[x>0]

#### Finding the Lot Size

In [25]:
lot_size = []
for i, itm in enumerate(df['Public Facts']):
    if len(itm.split('Lot Size')) > 1:
        temp = itm.split('Lot Size')[1]
        if 'Sq. Ft.' in temp:
            temp = temp.split('Sq. Ft.')[0]
            temp = re.sub("[^0-9.]+", '', temp)
            lot_size.append(temp)
        elif 'Acres' in temp:
            temp = temp.split('Acres')[0]
            temp = np.round(float(temp)*40000)
            lot_size.append(temp)
        else:
            lot_size.append('MI')
    else:
        lot_size.append('MI')
        
print(np.sum([x=='MI' for x in lot_size]))

330


In [26]:
##np.max([float(x) for x in lot_size if x != 'MI'])

In [27]:
##lot_size.index(992400.0)

In [28]:
##df['Public Facts'][2805]

In [29]:
##sns.distplot([float(x) for x in lot_size if x != 'MI'])

In [30]:
#lot_size = [re.sub("[.]{2}", '', x) for x in lot_size]
#lot_size = [np.round(float(x)*40000) if ((x!='MI') and (float(x) <= 10)) else x for x in lot_size]
##lot_size = [float(x) if (x!='') else 'MI' for x in lot_size]

##lot_size[0:10]

#### Finding the date the house relisted

In [31]:
d_listed = df['Price History']
d_listed.fillna("MI", inplace = True)
d_listed = [x.split('DateListed')[0] if (('DateListed' in x)) else 'MI' for x in d_listed]
d_listed = [x[::-1] for x in d_listed]
d_listed = [x.split('ecirP')[0] for x in d_listed]
d_listed = [x[::-1] for x in d_listed]

temp=[]
for i, itm in enumerate(d_listed):
    
    if itm != 'MI':
        d = re.search('[a-zA-Z]{3}( )[0-9]{1,2}(, )[0-9]{4}', itm)
        #print(i,d.group(0))
        temp.append(datetime.strptime(d.group(0), '%b %d, %Y').strftime('%Y-%m-%d'))
    else:
        temp.append(itm)

d_listed = temp
del temp  

In [32]:
print(np.sum([x == 'MI' for x in d_listed]), len(np.unique(d_listed)))

573 594


#### Finding the Sale Date

In [33]:
sale_date = df['Price History']
sale_date.fillna("MI", inplace = True)
sale_date = [x.split('Today')[1] if ('Today' in x) else 'MI' for x in sale_date]
sale_date = [x.split('DateSold')[0] if ('DateSold' in x) else 'MI' for x in sale_date]

temp=[]
for i, itm in enumerate(sale_date):
    
    if itm != 'MI':
        d = re.search('[a-zA-Z]{3}( )[0-9]{1,2}(, )[0-9]{4}', itm)
        ##print(i,d.group(0))
        temp.append(datetime.strptime(d.group(0), '%b %d, %Y').strftime('%Y-%m-%d'))
    else:
        temp.append(itm)

sale_date = temp
del temp

print(np.sum([x == 'MI' for x in sale_date]), len(np.unique(sale_date)))

328 300


#### Finding the number of times the house was delisted/relisted

In [34]:
times_delisted = df['Price History']
times_delisted.fillna("MI", inplace = True)
times_delisted = [(len(x.split('Delisted')) - 1) if (('Delisted' in x)) else 0 for x in times_delisted]
times_delisted[0:10]

[1, 1, 0, 3, 1, 5, 1, 0, 0, 2]

In [35]:
np.sum([x == 0 for x in times_delisted])

648

#### Finding the property size of the house

In [36]:
size = df.Size
size.fillna('MI', inplace=True)

prop_size = []
for i, itm in enumerate(df['Public Facts']):
    if size[i] != 'MI':
        prop_size.append(re.sub("[^0-9.]+", '', size[i]))
    else:
        
        if len(itm.split('Sq. Ft.')) > 1:
            temp = itm.split('Sq. Ft.')[1]
            if 'Stories' in temp:
                temp = temp.split('Stories')[0]
                temp = re.sub("[^0-9.]+", '', temp)
                if temp == '':
                    prop_size.append('MI')
                else:
                    prop_size.append(temp)
            else:
                prop_size.append('MI')
        else:
            prop_size.append('MI')
        
np.sum([x == 'MI' for x in prop_size])

221

In [37]:
##prop_size[0:10]

In [38]:
c_add = [' '.join([str(a),str(b),str(c),str(d)]) 
             for (a,b,c,d) in zip(df['Address'], df['City'], df['State'], df['Zip'])]

### Build the output dataset with all the additional features that were derived for each property (if those information were available. Otherwise the corresponding cell was popilated as "MI"

In [39]:
new_df = df[['property-href', ' Sale Price ']]
new_df.fillna('MI', inplace = True)


new_df.loc[:,'full_add'] = c_add
new_df.loc[:,'Zip_5'] = Zip_5
new_df.loc[:,'Zip_E'] = Zip_E
new_df.loc[:,'MLS'] = MLS

new_df.loc[:,'Beds'] = beds
new_df.loc[:,'Baths'] = baths

new_df.loc[:,'lot_size'] = lot_size
new_df.loc[:,'prop_size'] = prop_size

new_df.loc[:,'Age'] = age
new_df.loc[:,'Date_Listed'] = d_listed
new_df.loc[:,'Sale_Date'] = sale_date

new_df.loc[:,'Times_Delisted'] = times_delisted

new_df.loc[:,'floor_upg_sts'] = floor_upg_sts

new_df.loc[:,'stall_shower'] = stl_shr

#new_df.loc[:,'gourmet_kitchen'] = grmt_kit
#new_df.loc[:,'kitchen_island'] = kit_ilnd 
#new_df.loc[:,'kitchen_counter_top'] = k_ctr_top 
#new_df.loc[:,'wet_dry_bar'] = w_d_bar 
new_df.loc[:,'kitchen_upg_sts'] = kitchen_upg_sts

#new_df.loc[:,'recess_light'] = rcs_ltng 
#new_df.loc[:,'sky_light'] = sky_lt 
#new_df.loc[:,'crown_moulding'] = crn_mldg 
##new_df.loc[:,'fireplace'] = fireplace 
new_df.loc[:,'house_int_upg_sts'] = house_int_upg_sts

#new_df.loc[:,'sprinkler_sys'] = sprnklr_s 
#new_df.loc[:,'fencing'] = fencing 
##new_df.loc[:,'deck_patio'] = deck_patio 
new_df.loc[:,'house_ext_upg_sts'] = house_ext_upg_sts

new_df.loc[:,'townhouse'] = twn_house 

new_df.loc[:,'basement'] = bsmt
new_df.loc[:,'stories'] = stories

new_df.columns = ['property_href', 'sale_price', 'full_add', 'zip_5', 'zip_E', 'MLS',
       'beds', 'baths', 'lot_size', 'prop_size', 'age', 'date_listed',
       'sale_date', 'times_delisted', 'floor_upg_sts', 'stall_shower',
       'kitchen_upg_sts', 'house_int_upg_sts', 'house_ext_upg_sts',
       'townhouse', 'basement', 'stories']

#new_df.apply(lambda x: sum(x.isnull()))

new_df.columns



/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/dipanjanpaul/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

Index(['property_href', 'sale_price', 'full_add', 'zip_5', 'zip_E', 'MLS',
       'beds', 'baths', 'lot_size', 'prop_size', 'age', 'date_listed',
       'sale_date', 'times_delisted', 'floor_upg_sts', 'stall_shower',
       'kitchen_upg_sts', 'house_int_upg_sts', 'house_ext_upg_sts',
       'townhouse', 'basement', 'stories'],
      dtype='object')

In [41]:
new_df.shape

(3472, 22)

In [42]:
## Remove all new constructions from the dataset for better fit
new_df = new_df[new_df.age > 4]
new_df.shape

(3439, 22)

In [42]:
new_df.to_csv("../data/interim/listings.csv", header=True, index=False)

In [45]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://localhost/TB')

In [46]:
new_df.to_sql('listings', con=engine, index=False, if_exists='replace')

In [46]:
from pandas_profiling import ProfileReport
r_df = new_df.dropna()
prof = ProfileReport(r_df)
prof.to_file(output_file='../reports/listing_analysis.html')

In [81]:
new_df.apply(lambda x: sum(x=='MI'))

property_href           0
sale_price            170
full_add                0
zip_5                   0
zip_E                2066
MLS                   626
beds                 1577
baths                1559
lot_size              321
prop_size             220
age                     0
date_listed           568
sale_date             326
times_delisted          0
floor_upg_sts           0
stall_shower            0
kitchen_upg_sts         0
house_int_upg_sts       0
house_ext_upg_sts       0
townhouse               0
basement                0
stories                 0
dtype: int64

In [82]:
x = new_df[((new_df.lot_size != 'MI') & (new_df.prop_size != 'MI'))].copy()
x[(x.prop_size.astype('float') > x.lot_size.astype('float'))].shape

(176, 22)

## Prep the data for feeding the cluster creation model. The cluster-wise regression model utilizes an iterative approach to solve the multiple population problem.

In [83]:
##r_cols = ['Redfin_Estimate', 'zip_E', 'MLS', 'date_listed', 'sale_date', 'times_delisted', 'full_add']
r_cols = ['Redfin_Estimate', 'zip_E', 'MLS', 'date_listed', 'sale_date', 'times_delisted',
          'date_listed', 'sale_date', 'times_delisted', 'floor_upg_sts', 'stall_shower',
          'kitchen_upg_sts', 'house_int_upg_sts', 'house_ext_upg_sts',
          'townhouse', 'basement', 'stories']
k_cols = [x for x in new_df.columns if x not in r_cols]
new_df[k_cols].apply(lambda x: sum(x=='MI'))

property_href       0
sale_price        170
full_add            0
zip_5               0
beds             1577
baths            1559
lot_size          321
prop_size         220
age                 0
dtype: int64

In [84]:
r_df = new_df[k_cols].copy()
r_df['r_mi_cnt'] = r_df.apply(lambda x: sum(x=='MI'), axis=1)

r_df = r_df[r_df.r_mi_cnt == 0]
r_df.drop('r_mi_cnt', axis=1, inplace=True)
r_df.shape

(1643, 9)

In [85]:
s_p = [re.sub('[^0-9]', '', str(x)) for x in r_df.sale_price]
r_df.sale_price = s_p

In [86]:
r_df.zip_5 = r_df.zip_5.astype('category')
r_df.prop_size = r_df.prop_size.astype('float')
r_df.beds = r_df.beds.astype('float')
r_df.baths = r_df.baths.astype('float')
r_df.lot_size = r_df.lot_size.astype('float')
r_df.prop_size = r_df.prop_size.astype('float')
r_df.age = r_df.age.astype('float')
r_df.sale_price = r_df.sale_price.astype('float')

r_df.dtypes

property_href      object
sale_price        float64
full_add           object
zip_5            category
beds              float64
baths             float64
lot_size          float64
prop_size         float64
age               float64
dtype: object

In [87]:
r_df[(r_df.lot_size <= 60000)].shape

(1505, 9)

In [88]:
##r_df  = r_df[(r_df.lot_size > r_df.prop_size) & (r_df.lot_size > 3000)]
r_df = r_df[((r_df.sale_price >= 225000) & (r_df.sale_price <= 800000))]
r_df = r_df[r_df.prop_size <= 4000]
r_df = r_df[(r_df.beds <= 5) & (r_df.beds > 0)]
#r_df = r_df[(r_df.baths <= 4) & (r_df.baths > 1)]
r_df = r_df[(r_df.lot_size <= 60000)]
r_df = r_df[(r_df.age <= 90)]

df.reset_index(inplace=True, drop=True)
r_df.shape

(1057, 9)

In [89]:
for itm in r_df.dtypes[r_df.dtypes == 'object'].index:
    cat_dtype = pd.api.types.CategoricalDtype(categories=np.unique(r_df[itm]))
    r_df[itm] = r_df[itm].astype(cat_dtype)

r_df.dtypes

property_href    category
sale_price        float64
full_add         category
zip_5            category
beds              float64
baths             float64
lot_size          float64
prop_size         float64
age               float64
dtype: object

In [90]:
from pandas_profiling import ProfileReport
prof = ProfileReport(r_df)
prof.to_file(output_file='../reports/model_input_analysis.html')

In [91]:
##r_df.stories[r_df.stories == '1.5'] = '2'

In [92]:
x = r_df.zip_5.value_counts()
x

08873    235
08807    176
08844    153
08876     65
08540     49
07920     48
08835     40
08502     33
08805     31
07059     27
07924     23
08558     22
08812     20
07069     18
07921     18
07060     17
07063     17
08836     13
08880     11
08853     11
08869     10
08823      8
07062      7
08553      3
07934      1
07977      1
08821      0
07922      0
08528      0
07931      0
Name: zip_5, dtype: int64

In [93]:
model_df = r_df[r_df.zip_5.isin(x[x>20].index)].copy()
model_df.shape

(902, 9)

In [94]:
len(np.unique(model_df.property_href))

902

In [95]:
model_df.to_csv("../data/interim/clustering_model_input.csv", header=True, index=False)